# Klassifikation von Pistazien

In [67]:
"""Importieren der benötigten Bibliotheken"""
import pandas as pd
import arff
import numpy as np
import matplotlib.pyplot as plt

In [68]:
def check_requirements():
    """This function checks if all required files are present in the data folder."""
    import os

    print('Überprüfe ob alle benötigten Dateien vorhanden sind...')

    req_files = [
        "data/Pistachio_16_Features_Dataset_Citation_Request.txt",
        "data/Pistachio_16_Features_Dataset.arff",
        "data/Pistachio_16_Features_Dataset.xls",
    ]

    # Sicherstellen, dass der Ordner "data" existiert
    if not os.path.isdir('data'):
        raise AssertionError('Der Ordner "data" existiert nicht.')

    # Sicherstellen, dass alle benötigten Dateien im Ordner "data" existieren
    for file in req_files:
        if not os.path.isfile(file):
            raise AssertionError(f'Die Datei "{file}" existiert nicht.')

    print('... Alle benötigten Dateien sind vorhanden.')
    print('Viel Spaß!')
    pass

In [69]:
check_requirements()

Überprüfe ob alle benötigten Dateien vorhanden sind...
... Alle benötigten Dateien sind vorhanden.
Viel Spaß!


## Einführung in den Datensatz
Es wird ein Datensatz benutzt welcher in dem [Progress of Nutrition Vol. 23 No. 2 (2021)](https://doi.org/10.23751/pn.v23i2.9686) veröffentlicht wurde.<br>
Der Datensatz wurde in dem Paper [Classification of Pistachio Species Using Improved k-NN Classifier](https://www.mattioli1885journals.com/index.php/progressinnutrition/article/view/9686/9178) erstellt.

In dem Datensatz gibt es sowohl eine ausführung mit 16 Features als auch eine mit 28 Features. 
- <b>In diesem Projekt wird die Version mit 16 Features verwendet.</b>
    <br>(Die restlichen 12 Features sind "Farben Merkmale" und werden in diesem Projekt nicht verwendet.)

- <b>Morphological Features (12 Features):</b>
    1. Area
    2. Perimeter
    3. Major_Axis
    4. Minor_Axis
    5. Eccentricity
    6. Eqdiasq
    7. Solidity
    8. Convex_Area
    9. Extent
    10. Aspect_Ratio
    11. Roundness
    12. Compactness
- <b>Shape Features (4 Features)</b>
    13. Shapefactor_1
    14. Shapefactor_2
    15. Shapefactor_3
    16. Shapefactor_4


## Vorbereitung des Datensatzes
Um mit dem Datensatz besser arbeiten zu können, werden folgende Schritte durchlaufen:
- Es wird sich zuerst angeschaut wie der Datensatz aufgebaut ist. → [Explorative Datenanalyse](#Explorative-Datenanalyse)
- Anschließend muss der Datensatz ggf. bereinigt/aufbereitet werden. → [Aufbereitung der Daten](#Aufbereitung-der-Daten)
- Zuletzt wird der Datensatz noch in Trainings- und Testdaten aufgeteilt. → [Aufteilung der Daten in Trainings- und Testdaten](#Aufteilung-der-Daten-in-Trainings--und-Testdaten)

Danach kann es mit der Klassifikation losgehen. → [Klassifikation](#Klassifikation)

In [70]:
# Laden der Daten aus dem Excel-File in ein Pandas DataFrame
file_path = 'data/Pistachio_16_Features_Dataset.xls'
data_frame = pd.read_excel(file_path)

rows = data_frame.shape[0]
columns = data_frame.shape[1] - 1  # Klassifikationsspalte wird nicht mitgezählt


In [86]:
def print_descriptions():
    """Bei Bedarf kann diese Funktion aufgerufen werden, um die Beschreibungen der Attribute auzugeben."""
    
    # Anzeigen der ersten 5 Zeilen des Datensatzes, darunter die Namen der Attribute und die ersten 4 Datensätze
    print(data_frame.head(), '\n')
    print(f"Anzahl der Attribute: {columns}")
    print(f"Anzahl der Datensätze: {rows}")
    
    print('\n', data_frame.Class.value_counts(), '\n')
    print(data_frame.AREA.describe(), '\n')
    print(data_frame.PERIMETER.describe(), '\n')
    print(data_frame.MAJOR_AXIS.describe(), '\n')
    print(data_frame.MINOR_AXIS.describe(), '\n')
    print(data_frame.ECCENTRICITY.describe(), '\n')
    print(data_frame.EQDIASQ.describe(), '\n')
    print(data_frame.SOLIDITY.describe(), '\n')
    print(data_frame.CONVEX_AREA.describe(), '\n')
    print(data_frame.EXTENT.describe(), '\n')
    print(data_frame.ASPECT_RATIO.describe(), '\n')
    print(data_frame.ROUNDNESS.describe(), '\n')
    print(data_frame.COMPACTNESS.describe(), '\n')
    

In [87]:
print_descriptions()

    AREA  PERIMETER  MAJOR_AXIS  MINOR_AXIS  ECCENTRICITY   EQDIASQ  SOLIDITY  \
0  63391   1568.405    390.3396    236.7461        0.7951  284.0984    0.8665   
1  68358   1942.187    410.8594    234.7525        0.8207  295.0188    0.8765   
2  73589   1246.538    452.3630    220.5547        0.8731  306.0987    0.9172   
3  71106   1445.261    429.5291    216.0765        0.8643  300.8903    0.9589   
4  80087   1251.524    469.3783    220.9344        0.8823  319.3273    0.9657   

   CONVEX_AREA  EXTENT  ASPECT_RATIO  ROUNDNESS  COMPACTNESS  SHAPEFACTOR_1  \
0        73160  0.6394        1.6488     0.3238       0.7278         0.0062   
1        77991  0.6772        1.7502     0.2277       0.7181         0.0060   
2        80234  0.7127        2.0510     0.5951       0.6767         0.0061   
3        74153  0.7028        1.9879     0.4278       0.7005         0.0060   
4        82929  0.7459        2.1245     0.6425       0.6803         0.0059   

   SHAPEFACTOR_2  SHAPEFACTOR_3  SHAPE

### Explorative Datenanalyse
Die erste Frage, die sich stellt, lautet: <b>Wie ist der Datensatz aufgebaut?</b>

| Frage                                           | Ermittlung                               |                                               Antwort |
|:------------------------------------------------|------------------------------------------|------------------------------------------------------:|
| Wie viele Attribute gibt es?                    | `print(columns)`                         |                                                    16 |
| Wie viele Datensätze gibt es?                   | `print(rows)`                            |                                                  2148 |
| Wie viele Klassen gibt es?                      | `print(data_frame.Class.value_counts())` | <table><tr><th>Kirmizi</th><th>Siit</th></tr></table> |
| Wie viele Datensätze sind pro Klasse vorhanden? | `print(data_frame.Class.value_counts())` |     <table><tr><td>1232</td><td>916</td></tr></table> |

#### Wie sehen die Attribute insgesamt aus?

| Attribut     | Beschreibung                                                                                                                                                                  | Code                                 | Datentyp |          Min |           Max |         Mean |          Std |
|:-------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------|:---------|-------------:|--------------:|-------------:|-------------:|
| AREA         | Fläche                                                                                                                                                                        | `data_frame.AREA.describe()`         | float64  | 29808.000000 | 124008.000000 | 79950.655493 | 13121.737799 | 
| PERIMETER    | Umfang                                                                                                                                                                        | `data_frame.PERIMETER.describe()`    | float64  |   858.363000 |   2755.049100 |  1425.971751 |   375.565503 |
| MAJOR_AXIS   | Länge der großen Achse                                                                                                                                                        | `data_frame.MAJOR_AXIS.describe()`   | float64  |   320.344500 |    541.966100 |   446.248968 |   541.966100 |
| MINOR_AXIS   | Länge der kleinen Achse                                                                                                                                                       | `data_frame.MINOR_AXIS.describe()`   | float64  |   133.509600 |    383.046100 |   238.311842 |    30.310695 |
| ECCENTRICITY | Exzentrizität<br>Ist das Maß für die Abweichung<br> der Form von einem Kreis.<br>Normiert auf den Zahlenbereich<br>0 < e < 1                                                  | `data_frame.ECCENTRICITY.describe()` | float64  |     0.504900 |      0.946000 |     0.840219 |     0.048759 |
| EQDIASQ      | Äquivalenter Durchmesser quadriert<br>Ist ein Maß, mit dem nicht-kreisförmige<br> Objekte hinsichtlich der Fläche<br> mit einem Kreis verglichen werden.                      | `data_frame.EQDIASQ.describe()`      | float64  |   194.814600 |    397.356100 |   317.919173 |    26.908600 |
| SOLIDITY     | Solidität<br>Ist ein Maß, das verwendet wird,<br> um die Dichteverteilung oder die<br> Kompaktheit eines Objekts zu beschreiben.<br> Normiert auf den Zahlenbereich 0 < s < 1 | `data_frame.SOLIDITY.describe()`     | float64  |     0.588000 |      0.995100 |     0.940093 |     0.050452 |
| CONVEX_AREA  | Konvexe Fläche<br>Anzahl der Pixel im kleinsten<br/> konvexen Polygon, das die Fläche<br/> eines Bohnenkorns umfassen kann.                                                   | `data_frame.CONVEX_AREA.describe()`  | float64  | 37935.000000 | 132478.000000 | 85015.839851 | 13154.919327 |
| EXTENT       | Ausdehnung<br>Das Verhältnis der Fläche<br/>des Objekts zur Fläche<br/> des umschließenden Rechtecks.                                                                         | `data_frame.EXTENT.describe()`       | float64  |     0.427200 |      0.820400 |     0.716067 |     0.052532 | 
| ASPECT_RATIO | Seitenverhältnis<br>Verhältnis der Länge<br/>der großen Achse zur Länge<br/>der kleinen Achse.                                                                                | `data_frame.ASPECT_RATIO.describe()` | float64  |     1.158500 |      3.085800 |     1.898154 |     0.240100 |
| ROUNDNESS    | Rundheit<br>Ist das Maß, das beschreibt,<br/>wie sehr die Form eines Objekts<br/> einem perfekten Kreis ähnelt.                                                               | `data_frame.ROUNDNESS.describe()`    | float64  |     0.062800 |      0.933600 |     0.569172 |     0.212760 |
| COMPACTNESS  | Kompaktheit<br>                                                                     | `data_frame.COMPACTNESS.describe()`  | float64  |     0.476000 |      0.877900 |     0.713077 |     0.044547 |

count    2148.000000
mean        0.713077
std         0.044547
min         0.476000
25%         0.681550
50%         0.710700
75%         0.741650
max         0.877900
Name: COMPACTNESS, dtype: float64 


#### Wie sehen die Daten pro Klasse in einem Plot aus?

### Aufbereitung der Daten

### Aufteilung der Daten in Trainings- und Testdaten


## Klassifikation

### Künstliches Neuronales Netz (KNN)